## 1. PACOTES UTILIZADOS

In [1]:
from datetime import datetime
start = datetime.now()

from selenium import webdriver
from PIL import Image
import time
import io
import hashlib
import requests


## 2. DEFINIÇÃO DAS VARIAVEIS<br>

**DRIVER_PATH**: NOME/CAMINHO DO ARQUIVO EXE CORRESPONDENTE AO BROWSER A SER UTILIZADO (FIREFOX: firefoxdriver | CHROME: chromedriver)<br>
**SEARCH_TERM**: TERMO A SER PESQUISADO NO GOOGLE<br>
**NUMBER_IMAGES**: QUANTIDADE DE IMAGENS A SEREM BAIXADAS<br>
**TARGET_PATH**: LOCAL A SER SALVO AS IMAGENS BAIXADAS<br>

In [7]:
DRIVER_PATH = 'firefoxdriver.exe'
SEARCH_TERM = 'DRUGS'
NUMBER_IMAGES = 10
TARGET_PATH = r'D:\\Imagens\\PROJETO FINAL TESTE\\'

## 3. FUNÇÕES DE PESQUISA E DOWNLOAD DAS IMAGENS

In [3]:
def search_and_download(search_term:str,driver_path:str,target_path:str,number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Firefox(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [4]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"\nFound: {len(image_urls)} image links, done!")
                break
        else:
            print("\nFound:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [5]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"\nERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"\nSUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"\nERROR - Could not save {url} - {e}")

## 4. INÍCIO DA CHAMADA

In [8]:

search_and_download(
    search_term=SEARCH_TERM,
    driver_path=DRIVER_PATH,
    target_path=TARGET_PATH,
    number_images=100
)

end = datetime.now()
print('TEMPO TOTA\n\nL DE EXECUÇÃO: ', (end-start))

Found: 200 search results. Extracting links from 0:200

Found: 100 image links, done!

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTa8xPbIsK-QNn26_dU5KQ4YCBfqn0WGLnC7SmEfbhiM2TBQbDK&usqp=CAU - as D:\\Imagens\\PROJETO FINAL TESTE\\drugs\c5e40fa9d3.jpg

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSO4Wh-_YMY4H8juDzWmkWDnSST3DcQLDnpYurMs0zWlalwuQPx&usqp=CAU - as D:\\Imagens\\PROJETO FINAL TESTE\\drugs\cf489798bc.jpg

SUCCESS - saved https://d.newsweek.com/en/full/1537022/illegal-drugs.jpg - as D:\\Imagens\\PROJETO FINAL TESTE\\drugs\9f2b773ab0.jpg

SUCCESS - saved https://images.unsplash.com/photo-1471864190281-a93a3070b6de?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80 - as D:\\Imagens\\PROJETO FINAL TESTE\\drugs\93224641e7.jpg

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRLLMRjpSj77uMm9KQMzCI4F-VDdq2ouszfCs0_At_5YYfRkZCX&usqp=CAU - as D:\\Imagens\\PROJETO FINAL TESTE\\drugs\9b4308e932.jpg

SUCCESS -